import modules

In [29]:
import pandas as pd
import glob
import os
current_directory = os.getcwd()
print("현재 디렉토리:", current_directory)

현재 디렉토리: c:\apps\1stProject


목표 : 각 옵션종목을 각 dataframe으로 저장
dataframe이름 예로 201TB305는 콜옵션, 23년, 11월, 행사가 305이므로 -> TB305C
201TC2337는 콜옵션, 23년, 12월, 행사가 337.5이므로 -> TC337C
301TB312는 풋옵션, 23년, 11월, 행사가 312.5이므로 -> TB312P
301TC335는 풋옵션, 23년, 12월, 행사가 335이므로 -> TC335P

load files

In [30]:
strike_price = pd.read_csv('./old_dataset/strike_price.csv')

          일자  기준행사가    S-4    S-3    S-2    S-1    S+1    S+2    S+3    S+4
0   20231020  322.5  312.5  315.0  317.5  320.0  325.0  327.5  330.0  332.5
1   20231019  327.5  317.5  320.0  322.5  325.0  330.0  332.5  335.0  337.5
2   20231018  325.0  315.0  317.5  320.0  322.5  327.5  330.0  332.5  335.0
3   20231017  322.5  312.5  315.0  317.5  320.0  325.0  327.5  330.0  332.5
4   20231016  325.0  315.0  317.5  320.0  322.5  327.5  330.0  332.5  335.0
5   20231013  327.5  317.5  320.0  322.5  325.0  330.0  332.5  335.0  337.5
6   20231012  322.5  312.5  315.0  317.5  320.0  325.0  327.5  330.0  332.5
7   20231011  317.5  307.5  310.0  312.5  315.0  320.0  322.5  325.0  327.5
8   20231010  317.5  307.5  310.0  312.5  315.0  320.0  322.5  325.0  327.5
9   20231006  317.5  307.5  310.0  312.5  315.0  320.0  322.5  325.0  327.5
10  20231005  317.5  307.5  310.0  312.5  315.0  320.0  322.5  325.0  327.5
11  20231004  325.0  315.0  317.5  320.0  322.5  327.5  330.0  332.5  335.0
12  20230927

In [54]:
strike_price

,일자,기준행사가,S-4,S-3,S-2,S-1,S+1,S+2,S+3,S+4
0,20231020,322.5,312.5,315.0,317.5,320.0,325.0,327.5,330.0,332.5
1,20231019,327.5,317.5,320.0,322.5,325.0,330.0,332.5,335.0,337.5
2,20231018,325.0,315.0,317.5,320.0,322.5,327.5,330.0,332.5,335.0
3,20231017,322.5,312.5,315.0,317.5,320.0,325.0,327.5,330.0,332.5
4,20231016,325.0,315.0,317.5,320.0,322.5,327.5,330.0,332.5,335.0
5,20231013,327.5,317.5,320.0,322.5,325.0,330.0,332.5,335.0,337.5
6,20231012,322.5,312.5,315.0,317.5,320.0,325.0,327.5,330.0,332.5
7,20231011,317.5,307.5,310.0,312.5,315.0,320.0,322.5,325.0,327.5
8,20231010,317.5,307.5,310.0,312.5,315.0,320.0,322.5,325.0,327.5
9,20231006,317.5,307.5,310.0,312.5,315.0,320.0,322.5,325.0,327.5


In [31]:
folder_path = './option_price/' # 폴더 경로

In [32]:
combined_data = pd.DataFrame() # 합칠 CSV 파일들을 저장할 빈 DataFrame 생성
root_directory = './option_price/201TB305_352.5/' # 시작 디렉토리 설정

# root_directory와 그 하위 모든 디렉토리에서 CSV 파일을 찾아 합침
for root, dirs, files in os.walk(root_directory):
    for file in files:
        if file.endswith(".csv"):
            file_path = os.path.join(root, file)
            # CSV 파일을 DataFrame으로 읽어들임
            data = pd.read_csv(file_path)
            csv_file_name = os.path.basename(file_path)
            if csv_file_name.startswith('201'): # 201로 시작하면 끝에 C를 붙인다.
                csv_file_name = "C" + csv_file_name.split('.')[0].split('201')[1]
            elif csv_file_name.startswith('301'): # 301로 시작하면 끝에 P를 붙인다.
                csv_file_name = "P" + csv_file_name.split('.')[0].split('301')[1]
            data['체결시간'] = pd.to_datetime(data['체결시간'], format="%Y%m%d%H%M%S")
            data = data[['체결시간', '현재가']].rename(columns={'현재가': csv_file_name, '체결시간': 'date'})
            selected_rows = data[data['date'].dt.time == pd.to_datetime('09:30:00').time()]
            
            # 읽어들인 데이터를 기존 데이터프레임과 결합
            if combined_data.empty:
                combined_data = selected_rows
            else:
                combined_data = pd.merge(combined_data, selected_rows, on='date', how='outer')

# print(combined_data)


In [33]:
combined_data

,date,CTB305,CTB307,CTB310,CTB312,CTB315,CTB317,CTB319,CTB322,CTB324,CTB327,CTB332,CTB334,CTB337,CTB339,CTB342,CTB344,CTB347,CTB349,CTB352
0,2023-10-27 09:30:00,-6.45,-4.97,-3.70,2.65,-1.80,1.18,NaN,-0.43,NaN,-0.15,0.07,NaN,0.04,NaN,0.02,NaN,0.01,NaN,NaN
1,2023-10-26 09:30:00,8.96,7.16,5.60,4.25,3.11,2.17,NaN,0.92,NaN,0.34,0.13,NaN,0.06,NaN,0.03,NaN,0.02,NaN,0.01
2,2023-10-25 09:30:00,14.40,11.90,10.00,8.00,6.38,4.91,NaN,2.59,NaN,1.14,0.42,NaN,0.15,NaN,0.06,NaN,0.02,NaN,0.01
3,2023-10-24 09:30:00,-12.60,-10.60,-8.60,-7.00,-5.52,-4.20,NaN,-2.18,NaN,-0.94,-0.34,NaN,-0.12,NaN,-0.05,NaN,-0.02,NaN,0.01
4,2023-10-23 09:30:00,14.20,12.30,10.20,8.40,6.81,5.28,NaN,2.95,NaN,1.41,0.57,NaN,0.21,NaN,0.08,NaN,0.03,NaN,NaN
5,2023-10-20 09:30:00,-14.35,NaN,-10.55,-9.00,-7.15,-5.71,NaN,-3.33,NaN,-1.72,-0.77,NaN,0.31,NaN,0.12,NaN,0.05,NaN,-0.01
6,2023-10-13 09:30:00,22.80,NaN,NaN,15.40,NaN,11.80,NaN,7.93,NaN,5.04,2.92,NaN,1.50,NaN,0.67,NaN,0.26,NaN,0.10
7,2023-10-19 09:30:00,NaN,NaN,15.10,13.30,11.30,9.15,NaN,6.14,NaN,3.58,1.80,NaN,0.78,NaN,0.28,NaN,0.09,NaN,0.03
8,2023-10-18 09:30:00,NaN,NaN,-18.75,NaN,-14.00,NaN,NaN,-8.57,NaN,-5.41,-3.00,NaN,-1.43,NaN,-0.56,NaN,-0.18,NaN,-0.05
9,2023-10-16 09:30:00,NaN,NaN,16.05,NaN,NaN,10.40,NaN,6.47,NaN,3.90,2.03,NaN,0.91,NaN,0.34,NaN,0.13,NaN,0.05


콜옵션 23년 11월물 9시30분 현재가

In [34]:
CTB930 = pd.DataFrame() # 합칠 CSV 파일들을 저장할 빈 DataFrame 생성
root_directory = './option_price/201TB305_352.5/' # 시작 디렉토리 설정

# root_directory와 그 하위 모든 디렉토리에서 CSV 파일을 찾아 합침
for root, dirs, files in os.walk(root_directory):
    for file in files:
        if file.endswith(".csv"):
            file_path = os.path.join(root, file)
            # CSV 파일을 DataFrame으로 읽어들임
            data = pd.read_csv(file_path)
            csv_file_name = os.path.basename(file_path)
            if csv_file_name.startswith('201'): # 201로 시작하면 끝에 C를 붙인다.
                csv_file_name = "C" + csv_file_name.split('.')[0].split('201')[1]
            elif csv_file_name.startswith('301'): # 301로 시작하면 끝에 P를 붙인다.
                csv_file_name = "P" + csv_file_name.split('.')[0].split('301')[1]
            data['체결시간'] = pd.to_datetime(data['체결시간'], format="%Y%m%d%H%M%S")
            data = data[['체결시간', '현재가']].rename(columns={'현재가': csv_file_name, '체결시간': 'date'})
            selected_rows = data[data['date'].dt.time == pd.to_datetime('09:30:00').time()]
            
            # 읽어들인 데이터를 기존 데이터프레임과 결합
            if CTB930.empty:
                CTB930 = selected_rows
            else:
                CTB930 = pd.merge(CTB930, selected_rows, on='date', how='outer')
print(CTB930)


                  date  CTB305  CTB307  CTB310  CTB312  CTB315  CTB317  \
0  2023-10-27 09:30:00   -6.45   -4.97   -3.70    2.65   -1.80    1.18   
1  2023-10-26 09:30:00    8.96    7.16    5.60    4.25    3.11    2.17   
2  2023-10-25 09:30:00   14.40   11.90   10.00    8.00    6.38    4.91   
3  2023-10-24 09:30:00  -12.60  -10.60   -8.60   -7.00   -5.52   -4.20   
4  2023-10-23 09:30:00   14.20   12.30   10.20    8.40    6.81    5.28   
5  2023-10-20 09:30:00  -14.35     NaN  -10.55   -9.00   -7.15   -5.71   
6  2023-10-13 09:30:00   22.80     NaN     NaN   15.40     NaN   11.80   
7  2023-10-19 09:30:00     NaN     NaN   15.10   13.30   11.30    9.15   
8  2023-10-18 09:30:00     NaN     NaN  -18.75     NaN  -14.00     NaN   
9  2023-10-16 09:30:00     NaN     NaN   16.05     NaN     NaN   10.40   
10 2023-10-12 09:30:00     NaN     NaN   19.30     NaN   15.20  -13.15   
11 2023-10-11 09:30:00     NaN     NaN  -17.45     NaN  -13.15   11.45   
12 2023-10-10 09:30:00     NaN     NaN

In [35]:
CTB930['CTB324'].isna().value_counts()

CTB324
True    39
Name: count, dtype: int64

In [36]:
CTB930.head()

,date,CTB305,CTB307,CTB310,CTB312,CTB315,CTB317,CTB319,CTB322,CTB324,CTB327,CTB332,CTB334,CTB337,CTB339,CTB342,CTB344,CTB347,CTB349,CTB352
0,2023-10-27 09:30:00,-6.45,-4.97,-3.7,2.65,-1.80,1.18,NaN,-0.43,NaN,-0.15,0.07,NaN,0.04,NaN,0.02,NaN,0.01,NaN,NaN
1,2023-10-26 09:30:00,8.96,7.16,5.6,4.25,3.11,2.17,NaN,0.92,NaN,0.34,0.13,NaN,0.06,NaN,0.03,NaN,0.02,NaN,0.01
2,2023-10-25 09:30:00,14.40,11.90,10.0,8.00,6.38,4.91,NaN,2.59,NaN,1.14,0.42,NaN,0.15,NaN,0.06,NaN,0.02,NaN,0.01
3,2023-10-24 09:30:00,-12.60,-10.60,-8.6,-7.00,-5.52,-4.20,NaN,-2.18,NaN,-0.94,-0.34,NaN,-0.12,NaN,-0.05,NaN,-0.02,NaN,0.01
4,2023-10-23 09:30:00,14.20,12.30,10.2,8.40,6.81,5.28,NaN,2.95,NaN,1.41,0.57,NaN,0.21,NaN,0.08,NaN,0.03,NaN,NaN


콜 23년 11월 15시15분 현재가

In [37]:
CTB1515 = pd.DataFrame() # 합칠 CSV 파일들을 저장할 빈 DataFrame 생성
root_directory = './option_price/201TB305_352.5/' # 시작 디렉토리 설정

# root_directory와 그 하위 모든 디렉토리에서 CSV 파일을 찾아 합침
for root, dirs, files in os.walk(root_directory):
    for file in files:
        if file.endswith(".csv"):
            file_path = os.path.join(root, file)
            # CSV 파일을 DataFrame으로 읽어들임
            data = pd.read_csv(file_path)
            csv_file_name = os.path.basename(file_path)
            if csv_file_name.startswith('201'): # 201로 시작하면 끝에 C를 붙인다.
                csv_file_name = "C" + csv_file_name.split('.')[0].split('201')[1]
            elif csv_file_name.startswith('301'): # 301로 시작하면 끝에 P를 붙인다.
                csv_file_name = "P" + csv_file_name.split('.')[0].split('301')[1]
            data['체결시간'] = pd.to_datetime(data['체결시간'], format="%Y%m%d%H%M%S")
            data = data[['체결시간', '현재가']].rename(columns={'현재가': csv_file_name, '체결시간': 'date'})
            selected_rows = data[data['date'].dt.time == pd.to_datetime('15:15:00').time()]
            
            # 읽어들인 데이터를 기존 데이터프레임과 결합
            if CTB1515.empty:
                CTB1515 = selected_rows
            else:
                CTB1515 = pd.merge(CTB1515, selected_rows, on='date', how='outer')


                  date  CTB305  CTB307  CTB310  CTB312  CTB315  CTB317  \
0  2023-10-27 15:15:00    6.65    5.15    3.82    2.72    1.86    1.20   
1  2023-10-26 15:15:00   -7.15    5.64    4.29    3.15    2.23    1.52   
2  2023-10-25 15:15:00   13.15  -11.00   -9.08   -7.30   -5.68   -4.29   
3  2023-10-23 15:15:00   13.00  -10.85   -9.00   -7.39   -5.77   -4.44   
4  2023-10-24 15:15:00     NaN   12.60  -10.60   -8.66    7.11    5.58   
5  2023-10-20 15:15:00     NaN   12.80  -10.75    9.17   -7.40    5.89   
6  2023-10-17 15:15:00     NaN     NaN   18.55     NaN   14.20   12.10   
7  2023-10-13 15:15:00     NaN     NaN   17.55     NaN   13.35     NaN   
8  2023-10-06 15:15:00     NaN     NaN   12.80     NaN    9.25     NaN   
9  2023-10-05 15:15:00     NaN     NaN   12.35   10.55    9.00   -7.30   
10 2023-10-04 15:15:00     NaN     NaN  -12.70   10.95    9.10     NaN   
11 2023-09-04 15:15:00     NaN     NaN   31.50     NaN   26.90     NaN   
12 2023-10-19 15:15:00     NaN     NaN

In [53]:
CTB1515.head()

,date,CTB305,CTB307,CTB310,CTB312,CTB315,CTB317,CTB319,CTB322,CTB324,CTB327,CTB332,CTB334,CTB337,CTB339,CTB342,CTB344,CTB347,CTB349,CTB352
0,2023-10-27 15:15:00,6.65,5.15,3.82,2.72,1.86,1.20,NaN,-0.43,NaN,-0.15,-0.05,NaN,-0.02,NaN,0.02,NaN,NaN,NaN,NaN
1,2023-10-26 15:15:00,-7.15,5.64,4.29,3.15,2.23,1.52,NaN,0.63,NaN,0.24,0.11,NaN,0.05,NaN,0.03,NaN,0.01,NaN,0.01
2,2023-10-25 15:15:00,13.15,-11.00,-9.08,-7.30,-5.68,-4.29,NaN,-2.16,NaN,-0.89,0.31,NaN,0.11,NaN,0.05,NaN,0.02,NaN,0.01
3,2023-10-23 15:15:00,13.00,-10.85,-9.00,-7.39,-5.77,-4.44,NaN,-2.36,NaN,-1.06,-0.40,NaN,0.15,NaN,0.06,NaN,0.02,NaN,0.01
4,2023-10-24 15:15:00,NaN,12.60,-10.60,-8.66,7.11,5.58,NaN,3.10,NaN,-1.45,-0.56,NaN,-0.19,NaN,0.07,NaN,0.03,NaN,0.01


In [38]:
CTB1515.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39 entries, 0 to 38
Data columns (total 20 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    39 non-null     datetime64[ns]
 1   CTB305  4 non-null      float64       
 2   CTB307  6 non-null      float64       
 3   CTB310  12 non-null     float64       
 4   CTB312  10 non-null     float64       
 5   CTB315  15 non-null     float64       
 6   CTB317  12 non-null     float64       
 7   CTB319  0 non-null      float64       
 8   CTB322  18 non-null     float64       
 9   CTB324  0 non-null      float64       
 10  CTB327  21 non-null     float64       
 11  CTB332  21 non-null     float64       
 12  CTB334  0 non-null      float64       
 13  CTB337  26 non-null     float64       
 14  CTB339  0 non-null      float64       
 15  CTB342  27 non-null     float64       
 16  CTB344  0 non-null      float64       
 17  CTB347  30 non-null     float64       
 18  CTB349  0 no

콜 23년 12월물 9시30분 현재가

In [39]:
CTC930 = pd.DataFrame() # 합칠 CSV 파일들을 저장할 빈 DataFrame 생성
root_directory = './option_price/201TC305_337.5/' # 시작 디렉토리 설정

# root_directory와 그 하위 모든 디렉토리에서 CSV 파일을 찾아 합침
for root, dirs, files in os.walk(root_directory):
    for file in files:
        if file.endswith(".csv"):
            file_path = os.path.join(root, file)
            # CSV 파일을 DataFrame으로 읽어들임
            data = pd.read_csv(file_path)
            csv_file_name = os.path.basename(file_path)
            if csv_file_name.startswith('201'): # 201로 시작하면 끝에 C를 붙인다.
                csv_file_name = "C" + csv_file_name.split('.')[0].split('201')[1]
            elif csv_file_name.startswith('301'): # 301로 시작하면 끝에 P를 붙인다.
                csv_file_name = "P" + csv_file_name.split('.')[0].split('301')[1]
            data['체결시간'] = pd.to_datetime(data['체결시간'], format="%Y%m%d%H%M%S")
            data = data[['체결시간', '현재가']].rename(columns={'현재가': csv_file_name, '체결시간': 'date'})
            selected_rows = data[data['date'].dt.time == pd.to_datetime('09:30:00').time()]
            
            # 읽어들인 데이터를 기존 데이터프레임과 결합
            if CTC930.empty:
                CTC930 = selected_rows
            else:
                CTC930 = pd.merge(CTC930, selected_rows, on='date', how='outer')
print(CTC930)


                  date  CTC305  CTC307  CTC310  CTC312  CTC315  CTC320  \
0  2023-10-27 09:30:00  -10.55   -9.07   -7.75   -6.47    5.65   -3.63   
1  2023-10-26 09:30:00   13.00   11.05    9.80    8.32     NaN    4.88   
2  2023-09-26 09:30:00   26.60     NaN     NaN     NaN     NaN     NaN   
3  2023-10-23 09:30:00     NaN     NaN   13.50     NaN   10.60    7.85   
4  2023-10-20 09:30:00     NaN     NaN  -13.95     NaN  -11.00   -7.98   
5  2023-09-21 09:30:00     NaN     NaN   28.70     NaN   24.30     NaN   
6  2023-10-24 09:30:00     NaN     NaN     NaN  -10.75   -9.30   -7.03   
7  2023-10-04 09:30:00     NaN     NaN     NaN     NaN   12.80     NaN   
8  2023-04-06 09:30:00     NaN     NaN     NaN     NaN   23.00     NaN   
9  2023-03-29 09:30:00     NaN     NaN     NaN     NaN   20.20     NaN   
10 2023-10-25 09:30:00     NaN     NaN     NaN     NaN     NaN    7.35   
11 2023-10-10 09:30:00     NaN     NaN     NaN     NaN     NaN     NaN   
12 2023-10-06 09:30:00     NaN     NaN

콜옵션 23년 12월물 15시15분 현재가

In [40]:
CTC1515 = pd.DataFrame() # 합칠 CSV 파일들을 저장할 빈 DataFrame 생성
root_directory = './option_price/201TC305_337.5/' # 시작 디렉토리 설정

# root_directory와 그 하위 모든 디렉토리에서 CSV 파일을 찾아 합침
for root, dirs, files in os.walk(root_directory):
    for file in files:
        if file.endswith(".csv"):
            file_path = os.path.join(root, file)
            # CSV 파일을 DataFrame으로 읽어들임
            data = pd.read_csv(file_path)
            csv_file_name = os.path.basename(file_path)
            if csv_file_name.startswith('201'): # 201로 시작하면 끝에 C를 붙인다.
                csv_file_name = "C" + csv_file_name.split('.')[0].split('201')[1]
            elif csv_file_name.startswith('301'): # 301로 시작하면 끝에 P를 붙인다.
                csv_file_name = "P" + csv_file_name.split('.')[0].split('301')[1]
            data['체결시간'] = pd.to_datetime(data['체결시간'], format="%Y%m%d%H%M%S")
            data = data[['체결시간', '현재가']].rename(columns={'현재가': csv_file_name, '체결시간': 'date'})
            selected_rows = data[data['date'].dt.time == pd.to_datetime('15:15:00').time()]
            
            # 읽어들인 데이터를 기존 데이터프레임과 결합
            if CTC1515.empty:
                CTC1515 = selected_rows
            else:
                CTC1515 = pd.merge(CTC1515, selected_rows, on='date', how='outer')
print(CTC1515)


                  date  CTC305  CTC307  CTC310  CTC312  CTC315  CTC320  \
0  2023-10-26 15:15:00   11.25    9.80    8.49    7.16    6.00    4.09   
1  2023-10-20 15:15:00   17.90   16.30   14.30     NaN   11.05   -8.25   
2  2023-09-07 15:15:00   33.80     NaN   29.35     NaN     NaN     NaN   
3  2023-10-27 15:15:00     NaN    9.43    7.95     NaN    5.61   -3.68   
4  2023-10-23 15:15:00     NaN     NaN   12.90     NaN    9.79     NaN   
5  2023-10-04 15:15:00     NaN     NaN   15.50     NaN     NaN     NaN   
6  2023-08-02 15:15:00     NaN     NaN     NaN     NaN   34.25     NaN   
7  2023-07-05 15:15:00     NaN     NaN     NaN     NaN   30.90     NaN   
8  2023-10-25 15:15:00     NaN     NaN     NaN     NaN     NaN    6.97   
9  2023-10-24 15:15:00     NaN     NaN     NaN     NaN     NaN    7.98   
10 2023-10-19 15:15:00     NaN     NaN     NaN     NaN     NaN   10.80   
11 2023-10-17 15:15:00     NaN     NaN     NaN     NaN     NaN   13.60   
12 2023-10-16 15:15:00     NaN     NaN

풋옵션 23년 11월물 9시30분 현재가

In [41]:
PTB930 = pd.DataFrame() # 합칠 CSV 파일들을 저장할 빈 DataFrame 생성
root_directory = './option_price/301TB305_352.5/' # 시작 디렉토리 설정

# root_directory와 그 하위 모든 디렉토리에서 CSV 파일을 찾아 합침
for root, dirs, files in os.walk(root_directory):
    for file in files:
        if file.endswith(".csv"):
            file_path = os.path.join(root, file)
            # CSV 파일을 DataFrame으로 읽어들임
            data = pd.read_csv(file_path)
            csv_file_name = os.path.basename(file_path)
            if csv_file_name.startswith('201'): # 201로 시작하면 끝에 C를 붙인다.
                csv_file_name = "C" + csv_file_name.split('.')[0].split('201')[1]
            elif csv_file_name.startswith('301'): # 301로 시작하면 끝에 P를 붙인다.
                csv_file_name = "P" + csv_file_name.split('.')[0].split('301')[1]
            data['체결시간'] = pd.to_datetime(data['체결시간'], format="%Y%m%d%H%M%S")
            data = data[['체결시간', '현재가']].rename(columns={'현재가': csv_file_name, '체결시간': 'date'})
            selected_rows = data[data['date'].dt.time == pd.to_datetime('09:30:00').time()]
            
            # 읽어들인 데이터를 기존 데이터프레임과 결합
            if PTB930.empty:
                PTB930 = selected_rows
            else:
                PTB930 = pd.merge(PTB930, selected_rows, on='date', how='outer')
print(PTB930)


                  date  PTB305  PTB307  PTB310  PTB312  PTB315  PTB317  \
0  2023-10-27 09:30:00    3.42    4.42    5.64    7.06    8.70   10.65   
1  2023-10-26 09:30:00   -2.43   -3.18   -4.11   -5.23   -6.49   -8.02   
2  2023-10-25 09:30:00   -1.05   -1.44   -1.95   -2.61   -3.44   -4.45   
3  2023-10-24 09:30:00    1.63    2.14    2.78    3.57    4.55    5.72   
4  2023-10-23 09:30:00   -1.61   -2.07   -2.65   -3.35   -4.22   -5.25   
5  2023-10-20 09:30:00    2.00    2.50    3.12    3.85    4.75    5.83   
6  2023-10-19 09:30:00   -0.71   -0.93   -1.23   -1.61   -2.09   -2.70   
7  2023-10-18 09:30:00    0.34    0.46    0.62    0.83    1.13    1.52   
8  2023-10-17 09:30:00    0.50    0.68    0.93    1.25    1.65    2.17   
9  2023-10-16 09:30:00    0.76   -1.00   -1.31   -1.70   -2.20   -2.82   
10 2023-10-13 09:30:00    0.58    0.76    1.00    1.31    1.72    2.22   
11 2023-10-12 09:30:00    0.53    0.69    0.90    1.17    1.52    1.94   
12 2023-10-11 09:30:00   -0.74   -0.95

풋옵션 23년 11월물 15시15분 현재가

In [42]:
PTB1515 = pd.DataFrame() # 합칠 CSV 파일들을 저장할 빈 DataFrame 생성
root_directory = './option_price/301TB305_352.5/' # 시작 디렉토리 설정

# root_directory와 그 하위 모든 디렉토리에서 CSV 파일을 찾아 합침
for root, dirs, files in os.walk(root_directory):
    for file in files:
        if file.endswith(".csv"):
            file_path = os.path.join(root, file)
            # CSV 파일을 DataFrame으로 읽어들임
            data = pd.read_csv(file_path)
            csv_file_name = os.path.basename(file_path)
            if csv_file_name.startswith('201'): # 201로 시작하면 끝에 C를 붙인다.
                csv_file_name = "C" + csv_file_name.split('.')[0].split('201')[1]
            elif csv_file_name.startswith('301'): # 301로 시작하면 끝에 P를 붙인다.
                csv_file_name = "P" + csv_file_name.split('.')[0].split('301')[1]
            data['체결시간'] = pd.to_datetime(data['체결시간'], format="%Y%m%d%H%M%S")
            data = data[['체결시간', '현재가']].rename(columns={'현재가': csv_file_name, '체결시간': 'date'})
            selected_rows = data[data['date'].dt.time == pd.to_datetime('15:15:00').time()]
            
            # 읽어들인 데이터를 기존 데이터프레임과 결합
            if PTB1515.empty:
                PTB1515 = selected_rows
            else:
                PTB1515 = pd.merge(PTB1515, selected_rows, on='date', how='outer')
print(PTB1515)


                  date  PTB305  PTB307  PTB310  PTB312  PTB315  PTB317  \
0  2023-10-27 15:15:00   -3.04   -3.99   -5.14   -6.56   -8.23   10.05   
1  2023-10-26 15:15:00   -3.56   -4.52   -5.69   -7.04   -8.63  -10.35   
2  2023-10-25 15:15:00   -1.13    1.55   -2.10   -2.81    3.70    4.79   
3  2023-10-24 15:15:00    1.08   -1.45   -1.93    2.55    3.33    4.28   
4  2023-10-23 15:15:00    1.78    2.30    2.95    3.74    4.70    5.85   
5  2023-10-20 15:15:00    1.74    2.21    2.78    3.47    4.31    5.32   
6  2023-10-19 15:15:00   -0.92   -1.18   -1.53   -1.97   -2.51   -3.18   
7  2023-10-18 15:15:00    0.32    0.43    0.57    0.77    1.02    1.38   
8  2023-10-17 15:15:00    0.38   -0.51   -0.69   -0.92   -1.24   -1.65   
9  2023-10-16 15:15:00    0.80    1.06    1.38    1.79    2.30    2.94   
10 2023-10-13 15:15:00    0.58    0.77    1.01    1.33    1.74    2.26   
11 2023-10-12 15:15:00    0.47    0.61    0.80    1.05    1.37    1.77   
12 2023-10-11 15:15:00    0.86    1.11

In [43]:
PTB1515['PTB347'].isna().value_counts()

PTB347
True     48
False     1
Name: count, dtype: int64

풋 23년 12월물 9시30분 현재가

In [44]:
PTC930 = pd.DataFrame() # 합칠 CSV 파일들을 저장할 빈 DataFrame 생성
root_directory = './option_price/301TC305_337.5/' # 시작 디렉토리 설정

# root_directory와 그 하위 모든 디렉토리에서 CSV 파일을 찾아 합침
for root, dirs, files in os.walk(root_directory):
    for file in files:
        if file.endswith(".csv"):
            file_path = os.path.join(root, file)
            # CSV 파일을 DataFrame으로 읽어들임
            data = pd.read_csv(file_path)
            csv_file_name = os.path.basename(file_path)
            if csv_file_name.startswith('201'): # 201로 시작하면 끝에 C를 붙인다.
                csv_file_name = "C" + csv_file_name.split('.')[0].split('201')[1]
            elif csv_file_name.startswith('301'): # 301로 시작하면 끝에 P를 붙인다.
                csv_file_name = "P" + csv_file_name.split('.')[0].split('301')[1]
            data['체결시간'] = pd.to_datetime(data['체결시간'], format="%Y%m%d%H%M%S")
            data = data[['체결시간', '현재가']].rename(columns={'현재가': csv_file_name, '체결시간': 'date'})
            selected_rows = data[data['date'].dt.time == pd.to_datetime('09:30:00').time()]
            
            # 읽어들인 데이터를 기존 데이터프레임과 결합
            if PTC930.empty:
                PTC930 = selected_rows
            else:
                PTC930 = pd.merge(PTC930, selected_rows, on='date', how='outer')


In [45]:
PTC930.head()

,date,PTC305,PTC307,PTC310,PTC312,PTC315,PTC317,PTC320,PTC322,PTC325,PTC327,PTC330,PTC332,PTC335,PTC337
0,2023-10-27 09:30:00,6.62,7.66,8.77,9.99,11.45,-12.2,14.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-10-26 09:30:00,-5.61,-6.41,-7.39,-8.66,-10.00,NaN,-12.55,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-10-25 09:30:00,-3.54,-4.12,-4.80,-5.52,-6.50,-7.3,-8.45,-9.8,NaN,NaN,-13.85,NaN,NaN,NaN
3,2023-10-24 09:30:00,4.15,4.79,5.72,NaN,NaN,8.7,9.94,NaN,12.6,NaN,NaN,NaN,20.2,NaN
4,2023-10-23 09:30:00,-3.94,-4.62,-5.30,-6.13,-7.06,-8.2,NaN,NaN,NaN,-13.3,NaN,NaN,-18.4,NaN


In [46]:
PTC930['PTC337'].isna().value_counts()

PTC337
True    77
Name: count, dtype: int64

In [47]:
PTC930['PTC332'].isna().value_counts()

PTC332
True     75
False     2
Name: count, dtype: int64

풋 23년 12월물 15시15분 현재가

In [48]:
PTC1515 = pd.DataFrame() # 합칠 CSV 파일들을 저장할 빈 DataFrame 생성
root_directory = './option_price/301TC305_337.5/' # 시작 디렉토리 설정

# root_directory와 그 하위 모든 디렉토리에서 CSV 파일을 찾아 합침
for root, dirs, files in os.walk(root_directory):
    for file in files:
        if file.endswith(".csv"):
            file_path = os.path.join(root, file)
            # CSV 파일을 DataFrame으로 읽어들임
            data = pd.read_csv(file_path)
            csv_file_name = os.path.basename(file_path)
            if csv_file_name.startswith('201'): # 201로 시작하면 끝에 C를 붙인다.
                csv_file_name = "C" + csv_file_name.split('.')[0].split('201')[1]
            elif csv_file_name.startswith('301'): # 301로 시작하면 끝에 P를 붙인다.
                csv_file_name = "P" + csv_file_name.split('.')[0].split('301')[1]
            data['체결시간'] = pd.to_datetime(data['체결시간'], format="%Y%m%d%H%M%S")
            data = data[['체결시간', '현재가']].rename(columns={'현재가': csv_file_name, '체결시간': 'date'})
            selected_rows = data[data['date'].dt.time == pd.to_datetime('15:15:00').time()]
            
            # 읽어들인 데이터를 기존 데이터프레임과 결합
            if PTC1515.empty:
                PTC1515 = selected_rows
            else:
                PTC1515 = pd.merge(PTC1515, selected_rows, on='date', how='outer')
print(PTC1515)


                  date  PTC305  PTC307  PTC310  PTC312  PTC315  PTC317  \
0  2023-10-27 15:15:00   -6.38   -7.30   -8.49     NaN     NaN     NaN   
1  2023-10-26 15:15:00   -6.77    7.74    8.92   10.05   11.55     NaN   
2  2023-10-25 15:15:00    3.75     NaN    5.11     NaN    6.87    7.86   
3  2023-10-24 15:15:00    3.56    4.15    4.89    5.57    6.45    7.42   
4  2023-10-23 15:15:00   -4.33    5.04    5.78     NaN     NaN    8.70   
..                 ...     ...     ...     ...     ...     ...     ...   
63 2023-08-16 15:15:00     NaN     NaN     NaN     NaN     NaN     NaN   
64 2023-04-14 15:15:00     NaN     NaN     NaN     NaN     NaN     NaN   
65 2022-08-11 15:15:00     NaN     NaN     NaN     NaN     NaN     NaN   
66 2023-09-07 15:15:00     NaN     NaN     NaN     NaN     NaN     NaN   
67 2023-05-24 15:15:00     NaN     NaN     NaN     NaN     NaN     NaN   

    PTC320  PTC322  PTC325  PTC327  PTC330  PTC332  PTC335  PTC337  
0    14.10   15.90     NaN     NaN     NaN

In [49]:
PTC1515.head()

,date,PTC305,PTC307,PTC310,PTC312,PTC315,PTC317,PTC320,PTC322,PTC325,PTC327,PTC330,PTC332,PTC335,PTC337
0,2023-10-27 15:15:00,-6.38,-7.30,-8.49,NaN,NaN,NaN,14.10,15.90,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-10-26 15:15:00,-6.77,7.74,8.92,10.05,11.55,NaN,14.40,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-10-25 15:15:00,3.75,NaN,5.11,NaN,6.87,7.86,9.07,NaN,11.85,NaN,15.25,NaN,NaN,NaN
3,2023-10-24 15:15:00,3.56,4.15,4.89,5.57,6.45,7.42,8.55,9.76,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-10-23 15:15:00,-4.33,5.04,5.78,NaN,NaN,8.70,NaN,NaN,NaN,NaN,16.25,NaN,NaN,NaN


In [50]:
PTC1515['PTC332'].isna().value_counts()

PTC332
True     65
False     3
Name: count, dtype: int64

In [51]:
PTC1515['PTC335'].isna().value_counts()

PTC335
True     60
False     8
Name: count, dtype: int64

In [52]:
PTC1515['PTC337'].isna().value_counts()

PTC337
True     67
False     1
Name: count, dtype: int64